# TSFS12 Hand-in exercise 4: Collaborative control

Initial imports

In [1]:
import numpy as np
import math
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from collab_functions import CreateAgent, multi_agent_ode, formation_measurement_index, formation_references

Activate matplotlib for notebook. Plots must be in external windows due to animations.

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


# Define agent dynamics, control, and measurement functions

In [3]:
def f1(t, x, u, mdlpar):
    """Dynamic function for single integrator model
    
    Input arguments are:
       t - time 
       x - state vector of the agent (position) 
       u - the control input u, 
       mdlpar - dictionary with parameters of the model 
    
    Output:
       dxdt - state-derivative
    """
    return u

def g1(y, xref, ctrlpar):
    """Control function
    
    Compute the control signal, in this case it is a P-controller which gives a control signal 
    proportional to the vector between the current position and the reference position. 
    
    Input arguments:
      y - measurement y, 
      xref - the reference vector xref (in this case the desired position of the agent)
      ctrlpar - dictionary which contains parameters used by the controller (in this case the proportional gain k).

    Output argument:
      Control signal    
    """
    
    k = ctrlpar['k']
    return k*(xref-y)

def h1(x, measpar):
    """Measurement function

     The measurement function gives the measurements available to the
     agent. 
     
     Input arguments:
         x - the complete state vector of all agents in the multi-agent system
         measpar - dictionary containing parameters used in the function. In
                   this case the indices in the state vector x of the states measured by the
                   agent stored in key idx.

    Output:
        y - The measurement vector
    """    
    meas_idx = measpar['idx']
    return x[meas_idx]

def g4(y, xref, ctrlpar):

    k = ctrlpar['k']
    gamma_sum=np.array([0,0])   
    cmpt=math.floor((len(xref)/2))
    for i in range(cmpt-1):
        gamma=(((y[2*i]-y[0])**2+(y[2*i+1]-y[1])**2)-((xref[2*i]-xref[0])**2+(xref[2*i+1]-xref[1])**2))
        delta_pos=np.array([y[2*i]-y[0], y[2*i+1]-y[1]])
        gamma_sum=gamma_sum+4*gamma*delta_pos
    return k*gamma_sum

# Create all agents

Create all agents, specifying models, parameters, controllers, the measurement structure, and references for controllers.

In [4]:
modelparam = {'m': 1}
controller = {'k': 1}
controller_2 = {'k': 10}

simple_formation_measurement_index = [{'idx': [0, 1]},
                                      {'idx': [2, 3, 0, 1, 4, 5, 6, 7]},
                                      {'idx': [4, 5, 0, 1, 2, 3, 6, 7, 8, 9]},
                                      {'idx': [6, 7, 2, 3, 4, 5, 8, 9, 10, 11]},
                                      {'idx': [8, 9, 4, 5, 6, 7,10, 11, 12, 13]},
                                      {'idx': [10, 11, 6, 7, 8, 9 , 12, 13, 14, 15]},
                                      {'idx': [12, 13, 8, 9, 10, 11, 14, 15]},
                                      {'idx': [14, 15, 10, 11, 12, 13]}]

simple_formation_references = [lambda t: [0+np.cos(-t), 6+np.sin(-t)],
                               lambda t: [-1, 5, 0, 6, 1, 5, -1, 3],
                               lambda t: [1, 5, 0, 6, -1, 5, -1, 3, 1, 3],
                               lambda t: [-1, 3, -1, 5, 1, 5, 1, 3, -1, 1],
                               lambda t: [1, 3, 1, 5, -1, 3, -1, 1, 1, 1],
                               lambda t: [-1, 1, -1, 3, 1, 3, 1, 1, 0, 0],
                               lambda t: [1, 1, 1, 3, -1, 1, 0, 0],
                               lambda t: [0, 0, -1, 1, 1, 1]]

agents = [
    CreateAgent(f1, modelparam, g1, controller, h1, simple_formation_measurement_index[0], simple_formation_references[0]), 
    CreateAgent(f1, modelparam, g4, controller, h1, simple_formation_measurement_index[1], simple_formation_references[1]),
    CreateAgent(f1, modelparam, g4, controller, h1, simple_formation_measurement_index[2], simple_formation_references[2]),
    CreateAgent(f1, modelparam, g4, controller, h1, simple_formation_measurement_index[3], simple_formation_references[3]),
    CreateAgent(f1, modelparam, g4, controller, h1, simple_formation_measurement_index[4], simple_formation_references[4]), 
    CreateAgent(f1, modelparam, g4, controller, h1, simple_formation_measurement_index[5], simple_formation_references[5]),
    CreateAgent(f1, modelparam, g4, controller, h1, simple_formation_measurement_index[6], simple_formation_references[6]),
    CreateAgent(f1, modelparam, g4, controller, h1, simple_formation_measurement_index[7], simple_formation_references[7])
]
n = 2  # Number of states

# Simulate the multi-agent system

This part should not need to be modified except specifying the initial state.

In [5]:
x0 = np.array([0, 6, -1, 5, 1, 5, -1, 3, 1, 3, -1, 1, 1, 1, 0, 0])
t = np.arange(0, 10, 0.05)
x = odeint(lambda x, t: multi_agent_ode(x, t, agents, n), x0, t)

Animate the simulated agents.

In [6]:
fig = plt.figure(20, clear=True)
plt.axis([-2, 6, -3, 6])

m = tuple([plt.plot(np.nan, np.nan, 'bo')[0] for k in range(len(agents))])
def animate(i):
    for idx, mi in enumerate(m):
        mi.set_xdata(x[i, 0 + n*idx])
        mi.set_ydata(x[i, 1 + n*idx])
    return m

ani = animation.FuncAnimation(fig, animate, interval=1, frames=x.shape[0], blit=True, save_count=50, repeat=False)